In [ ]:
!pip install sentence-transformers

In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

In [5]:
h1 = model.encode("안녕하세요.")
h2 = model.encode("식사하셨나요.")
cosine_similarity([h1], [h2])

array([[0.3875957]], dtype=float32)

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/kairess/mental-health-chatbot/master/wellness_dataset_original.csv")
df.head(10)

In [ ]:
df = df.drop(columns=['Unnamed: 3'])
df.head()

In [ ]:
df = df.dropna()
df.head()

In [ ]:
df.loc[0,'유저']

In [ ]:
model.encode(df.loc[0,'유저'])

In [ ]:
df['embedding'] = pd.Series([[]] * len(df))
df['embedding'] = df['유저'].map(lambda x: list(model.encode(x)))
df.head()

In [18]:
text = '요즘 남편이 비트코인도 하고 너무 속을 썩이네요.'
embedding = model.encode(text)


In [ ]:
df['similarity'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())
df.head()

In [20]:
df['similarity'].idxmax()

2147

In [ ]:
df.loc[df['similarity'].idxmax()]

In [ ]:
answer = df.loc[df['similarity'].idxmax()]

print(answer['구분'])
print(answer['유저'])
print(answer['챗봇'])
print(answer['similarity'])

In [ ]:
import gradio as gr

def chatbot(user):
    embedding = model.encode(user)

    df['similarity'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())
    answer = df.loc[df['similarity'].idxmax()]
    return answer['챗봇']

demo = gr.Interface(fn=chatbot, inputs='text', outputs='text')
demo.launch(debug=True, share=True)
